# エラーへの対処・修正済みコード

『Statistical Rethinking A Bayesian Course with Examples in R and Stan 』は、  
ベイズ統計学の基礎的な内容を実用例を交えて紹介している非常に優れた教科書です。

教科書には分析で使用するRのコードが記載されていますが、  
著者のRichard McElreath先生のwebサイト  

https://xcelab.net/rm/statistical-rethinking/  

で、 同じ内容を他の言語/パッケージで行うためのGitHubが紹介されています。  
例えばpymcで再現したコードを公開しているリポジトリとして以下があります。  

pymc (https://github.com/pymc-devs/pymc-resources/tree/main)  

notebookの形式で教科書内のコードを(おそらくほぼ)すべて再現されています。  
博愛に満ちた素晴らしい成果物なのですが、一部エラーが出るコードがあります。  

このノートブックではエラーが出たコードの箇所を報告し、  修正できたところは修正済みのコードを記載しています。  

## Chapter5 (Chp_05.ipynb)

#### 5.33, 5.34

元のjupyter notebookのコードでは以降のコードでエラーが生じます。  

    shared_N = shared(dcc["N"].values)
    with pm.Model() as m5_5_draft:
       sigma = pm.Exponential("sigma", 1)
       bN = pm.Normal("bN", 0, 1)
       a = pm.Normal("a", 0, 1)
       mu = pm.Deterministic("mu", a + bN * shared_N)

       K = pm.Normal("K", mu, sigma, observed=dcc["K"])

       m5_5_draft_trace = pm.sample()

    xseq = [-2, 2]
    shared_N.set_value(np.array(xseq))
    with m5_5_draft:
       m5_5_draft_prior_predictive = pm.sample_prior_predictive()  
       
shared_NとKなどの形状が一致せずエラーが発生するようです。  
元のRコードではパッケージの関数を使用しているためどのように処理しているかわりません。  
よって以下ではxseqの形状に合わせてKの形状が変化するように修正しました。

In [ ]:
xseq = np.array([-2, 2])
shared_N.set_value(xseq)

with pm.Model() as m5_5_draft:
    sigma = pm.Exponential("sigma", 1)
    bN = pm.Normal("bN", 0, 1)
    a = pm.Normal("a", 0, 1)
    mu = pm.Deterministic("mu", a + bN * shared_N)

    # ここでobservedの部分を適切な形状に更新
    observed_values = dcc["K"].values[:len(xseq)] 
    K = pm.Normal("K", mu, sigma, observed=observed_values)

    m5_5_draft_trace = pm.sample()


In [ ]:
xseq = [-2, 2]
shared_N.set_value(np.array(xseq))
with m5_5_draft:
    m5_5_draft_prior_predictive = pm.sample_prior_predictive()

#### 5.35  
前のエラーと同様にデータの形状が合わずエラーが出ます。  
同じように修正しました。

In [ ]:
xseq = np.array([-2, 2])
shared_N.set_value(xseq)

with pm.Model() as m5_5:
    sigma = pm.Exponential("sigma", 1)
    bN = pm.Normal("bN", 0, 0.5)
    a = pm.Normal("a", 0, 0.2)
    mu = pm.Deterministic("mu", a + bN * shared_N)

    # ここでobservedの部分を適切な形状に更新
    observed_values = dcc["K"].values[:len(xseq)] 
    K = pm.Normal("K", mu, sigma, observed=observed_values)

    m5_5_trace = pm.sample()
m5_5_data = az.extract_dataset(m5_5_trace)